### This notebook is for IBM Data Science Course Capstone Project.

In [1]:
import numpy as np
import pandas as pd

### Week_3_1 (Basic Processing)

This is for acquire postal codes of Toronto.

In [2]:
df_toronto = pd.read_excel('./coursera_postal.xlsx',names=['Postcode','Borough','Neighborhood'])

In [3]:
df_toronto.head()

,Postcode,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


In [4]:
df_toronto.shape

(179, 3)

#### For exclude "Not assigned values"

In [5]:
df_toronto = df_toronto[(df_toronto.Borough.notnull())]
df_toronto = df_toronto[(df_toronto.Borough != "Not assigned")]

# Group by PostalCode/Borough
df_toronto = pd.DataFrame(df_toronto.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(list)).reset_index()
df_toronto['Neighborhood'] = df_toronto['Neighborhood'].apply(lambda x: ', '.join(x))

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_toronto.loc[ (df_toronto.Neighborhood.isnull() == True) |
               (df_toronto.Neighborhood == "Not assigned")
               , 'Neighborhood'] = df_toronto.Borough

df_toronto.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
df_toronto.shape

(103, 3)

### Week_3_2 (Connecting to geospatial Data)

In [7]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
# merge to previous DataFrame
lat_lon.rename(columns={'Postal Code':'Postcode'},inplace=True)
df3 = pd.merge(df_toronto,lat_lon,on='Postcode')
df3.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### week 4- 5  (Introduction/Business Problem)


- Analysing Area : Toronto, Canada

- Analysing Aim : After sementing the regions of Toronto into a couple of sub-segments, we will discover the features of each segments.

- Analysing Method :
            
     K-means, which is very powerful tool for clustering.
            
     It is one of tools for machine learing. By calculating similarities between items,
            
     it determines the possibilities to which categorries one elemtent will belong.
     

- Business Problem : 
            
     To make a decision on business, we need to know the business to be done will go through a 
            
     tough battle or have few rivals. In other words, knowing the categories of business is quite 
            
     essential to run a business. 
            
     For these reasons, clustering by k-means algorithm will give us very power insights.
     
                   
- Data Manipulation : Using Panda, sklearn- Kmeans, and so on


- Conclusion : It will be made at the end of this codes.
                    

week 4- 5 (Data Processing)

In [9]:
import folium
import requests

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [12]:
import geocoder 

##### geospatial data of Toronto

In [13]:
address = 'Toronto, ON'

# initialize to None
lat_lng_coords = None

# loop until we get the coordinates
while(lat_lng_coords is None):
    g = geocoder.arcgis('Toronto, Ontario')
    lat_lng_coords = g.latlng

latitude_toronto = lat_lng_coords[0]
longitude_toronto = lat_lng_coords[1]

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.648690000000045, -79.38543999999996.


##### making visualization of districts

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postalCode in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood'], df3['Postcode']):
    label = '{}'.format(postalCode)
    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto) 
    
map_toronto

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto_radius = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postalCode in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood'], df3['Postcode']):
    label = '{}'.format(postalCode)
    label = folium.Popup(label, parse_html=True)

    folium.Circle(
        [lat, lng],
        radius=1100,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3).add_to(map_toronto_radius) 
    
map_toronto_radius

#### Connecting to Foursquare API for venues information

In [20]:
CLIENT_ID = '4NVVCSEOGT412NZP1KJ20RCUFMRX4UZB4DNNUHF4M44DSUZW' # your Foursquare ID
CLIENT_SECRET = 'ZYUYGEL5Q223SSKDXBL1QLE2X4UY0FWN50VF4DS4NUMTSASB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Acquiring Venues information

In [21]:
def getNearbyVenues(postalCodes, boroughs, neighbourhoods, latitudes, longitudes):
    
    venues_list=[]
    # Loop through each neighbourhood given in parameters
    for postalCode, borough, neighbourhood, lat, lng in zip(postalCodes, boroughs, neighbourhoods, latitudes, longitudes):
            
        # create the API request URL to explore the neighbourhood using FoursquareAPI
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            1100, 
            100)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue : name, latitude, longitude, and the categories' names
        venues_list.append([(
            postalCode,
            borough,
            neighbourhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    # add the venues in the dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                        'PostalCode',
                        'Borough',
                        'Neighborhood', 
                        'Neighborhood Latitude', 
                        'Neighborhood Longitude', 
                        'Venue', 
                        'Venue Latitude', 
                        'Venue Longitude', 
                        'Venue Category'
    ]
    
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(  
                                    postalCodes=df3['Postcode'],
                                    boroughs=df3['Borough'],
                                    neighbourhoods=df3['Neighborhood'],
                                    latitudes=df3['Latitude'],
                                    longitudes=df3['Longitude']
                                  )

toronto_venues.head()

,PostalCode,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
2,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,RBC Royal Bank,43.798782,-79.197090,Bank
3,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
4,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store


#### Data organizing

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postalCode, borough, and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 
toronto_onehot['Borough'] = toronto_venues['Borough'] 

toronto_onehot.pop('Neighborhood')
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move postalCode, borough, neighborhood column to the first column
for i in range(0, 3):
    toronto_onehot = toronto_onehot[[toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])]

toronto_onehot.head()

,PostalCode,Borough,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_grouped = toronto_onehot.groupby(['PostalCode','Borough', 'Neighborhood']).mean().reset_index()
toronto_grouped.head()

,PostalCode,Borough,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,M1B,Scarborough,"Malvern, Rouge",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.045455,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0
3,M1G,Scarborough,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0
4,M1H,Scarborough,Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.025,0.0,0.025,0.0


#### Most Common Vanues in each district

In [26]:

num_top_venues = 5

# Iterate through all the grouped dataframe
for index, row in toronto_grouped.iterrows():
    tempPostalCode = row['PostalCode']
    tempBorough = row['Borough']
    tempNeighborhood = row['Neighborhood']
    
    print("----"+tempPostalCode + " / " + tempBorough + " / " + tempNeighborhood +"----")
    
    # Create a temp df filtered on the current neighbourhood (key : postal code x borouhg x neighbourhood)
    temp = toronto_grouped[
        (toronto_grouped.PostalCode == tempPostalCode) &
        (toronto_grouped.Borough == tempBorough) &
        (toronto_grouped.Neighborhood == tempNeighborhood)
    ].T.reset_index()
    
    temp.columns = ['venue','freq']

    # We skip the key PostalCode x Borough x Neighbourhood : length = 3, iloc[3:]
    temp = temp.iloc[len(['PostalCode', 'Borough', 'Neighborhood']):]
    temp['freq'] = temp['freq'].astype(float)
    
    # Round the frequency with two digits
    temp = temp.round({'freq': 2})
    
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B / Scarborough / Malvern, Rouge----
                  venue  freq
0  Fast Food Restaurant  0.14
1           Coffee Shop  0.09
2                 Trail  0.09
3            Hobby Shop  0.05
4        Cosmetics Shop  0.05


----M1C / Scarborough / Rouge Hill, Port Union, Highland Creek----
                venue  freq
0        Burger Joint   0.2
1  Italian Restaurant   0.2
2      Breakfast Spot   0.2
3          Playground   0.2
4                Park   0.2


----M1E / Scarborough / Guildwood, Morningside, West Hill----
                  venue  freq
0           Pizza Place  0.12
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3                  Bank  0.08
4        Discount Store  0.04


----M1G / Scarborough / Woburn----
               venue  freq
0  Indian Restaurant  0.14
1        Coffee Shop  0.14
2        Pizza Place  0.14
3               Park  0.14
4   Department Store  0.07


----M1H / Scarborough / Cedarbrae----
               venue  freq
0        Coffee Shop  0.10
1  

                           venue  freq
0                           Café  0.09
1  Vegetarian / Vegan Restaurant  0.06
2                            Bar  0.06
3                    Coffee Shop  0.05
4                    Yoga Studio  0.03


----M5V / Downtown Toronto / CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0  Harbor / Marina  0.14
1      Coffee Shop  0.14
2             Café  0.10
3              Gym  0.10
4            Track  0.10


----M5W / Downtown Toronto / Stn A PO Boxes----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.07
2           Restaurant  0.05
3                Hotel  0.05
4  Japanese Restaurant  0.05


----M5X / Downtown Toronto / First Canadian Place, Underground city----
                 venue  freq
0          Coffee Shop  0.09
1                Hotel  0.06
2                 Café  0.05
3  Japanese Restaurant  0.04
4           Restaurant  0.04




In [31]:
def return_most_common_venues(row, num_top_venues):
    # Remove the key PostalCode x Borough x Neighbourhood from the row
    row_categories = row.iloc[len(['PostalCode', 'Borough', 'Neighborhood']):]
    
    # Sort ascending
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    # Return the top num_top_venues
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode', 'Borough', 'Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe, and set it with the columns names
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)

# add the keys from the grouped dataframe (Postal code x Borough x Neighborhood)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

# loop through each rows
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, len(['PostalCode', 'Borough', 'Neighborhood']):] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",Fast Food Restaurant,Coffee Shop,Trail,Arts & Crafts Store,Greek Restaurant,Park,Chinese Restaurant,Bank,Bakery,Supermarket
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",Playground,Burger Joint,Italian Restaurant,Breakfast Spot,Park,Zoo,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",Pizza Place,Fast Food Restaurant,Bank,Coffee Shop,Fried Chicken Joint,Automotive Shop,Food & Drink Shop,Sports Bar,Smoothie Shop,Breakfast Spot
3,M1G,Scarborough,Woburn,Indian Restaurant,Park,Coffee Shop,Pizza Place,Chinese Restaurant,Sandwich Place,Department Store,Mobile Phone Shop,Juice Bar,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,Coffee Shop,Gas Station,Indian Restaurant,Pharmacy,Sandwich Place,Bank,Bakery,Deli / Bodega,Fast Food Restaurant,Chinese Restaurant


In [34]:
from sklearn.cluster import KMeans

#### Clustering By K-means algorithm

In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering = toronto_grouped_clustering.drop('PostalCode', 1)
toronto_grouped_clustering = toronto_grouped_clustering.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 2, 4, 4, 4, 4, 4, 4, 0, 2])

In [36]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [39]:
toronto_merged = df3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index(['PostalCode','Borough', 'Neighborhood']), on=['Postcode','Borough', 'Neighborhood'])

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Coffee Shop,Trail,Arts & Crafts Store,Greek Restaurant,Park,Chinese Restaurant,Bank,Bakery,Supermarket
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Playground,Burger Joint,Italian Restaurant,Breakfast Spot,Park,Zoo,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Pizza Place,Fast Food Restaurant,Bank,Coffee Shop,Fried Chicken Joint,Automotive Shop,Food & Drink Shop,Sports Bar,Smoothie Shop,Breakfast Spot
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Indian Restaurant,Park,Coffee Shop,Pizza Place,Chinese Restaurant,Sandwich Place,Department Store,Mobile Phone Shop,Juice Bar,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Coffee Shop,Gas Station,Indian Restaurant,Pharmacy,Sandwich Place,Bank,Bakery,Deli / Bodega,Fast Food Restaurant,Chinese Restaurant


## Insights (Conclusion)

I have made clustering on regions of Toronto city into 5 sub-categories.

The most predominant venues are like below.

By the results, we can gain some insights into business models.


In [43]:
toronto_merged[toronto_merged['Cluster Labels'] == 0].head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0,Pizza Place,Ice Cream Shop,Harbor / Marina,Beach,Sports Bar,Grocery Store,Park,Discount Store,Hardware Store,Sandwich Place
31,M3L,North York,Downsview,43.739015,-79.506944,0,Shopping Mall,Grocery Store,Park,Vietnamese Restaurant,Pizza Place,Bank,Farmers Market,Escape Room,Ethiopian Restaurant,Event Space
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,0,Pizza Place,Grocery Store,Sushi Restaurant,Hardware Store,Bus Line,Fried Chicken Joint,Liquor Store,Beer Store,Pharmacy,Fast Food Restaurant


In [44]:
toronto_merged[toronto_merged['Cluster Labels'] == 1].head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493,1,Korean Restaurant,Coffee Shop,Café,Middle Eastern Restaurant,Pizza Place,Diner,Bank,Sushi Restaurant,Supermarket,Beer Store
27,M3C,North York,Don Mills,43.725900,-79.340923,1,Restaurant,Coffee Shop,Gym,Pizza Place,Intersection,Café,Sporting Goods Shop,Supermarket,Japanese Restaurant,Beer Store
30,M3K,North York,Downsview,43.737473,-79.464763,1,Coffee Shop,Turkish Restaurant,Electronics Store,Athletics & Sports,Soccer Field,Café,Men's Store,Gas Station,Chinese Restaurant,Other Repair Shop
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Beach,Pizza Place,Coffee Shop,Breakfast Spot,Japanese Restaurant,Thai Restaurant,Grocery Store,BBQ Joint,Caribbean Restaurant
38,M4G,East York,Leaside,43.709060,-79.363452,1,Coffee Shop,Bakery,Grocery Store,Indian Restaurant,Bank,Sporting Goods Shop,Electronics Store,Sushi Restaurant,Sandwich Place,Furniture / Home Store
40,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,1,Coffee Shop,Café,Greek Restaurant,Pizza Place,Ethiopian Restaurant,Bakery,Fast Food Restaurant,Beer Bar,Bar,Bank
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Café,Pub,Bakery,Pizza Place,Italian Restaurant,Bank,Restaurant,Yoga Studio
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Indian Restaurant,Park,Coffee Shop,Grocery Store,Beach,Café,Pub,Restaurant,Butcher,Gym
43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Vietnamese Restaurant,Bakery,Brewery,Bar,American Restaurant,Diner,French Restaurant,Italian Restaurant,Café
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Gym / Fitness Center,Park,Coffee Shop,College Gym,Café,Bookstore,Trail,College Quad,Fast Food Restaurant,Event Space


In [45]:
toronto_merged[toronto_merged['Cluster Labels'] == 2].head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Playground,Burger Joint,Italian Restaurant,Breakfast Spot,Park,Zoo,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,2,Park,College Stadium,Ice Cream Shop,Thai Restaurant,Convenience Store,Gym,Restaurant,Auto Workshop,Gym Pool,Café
20,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714,2,Park,Gym / Fitness Center,Coffee Shop,Zoo,Escape Room,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321,2,Park,Restaurant,Liquor Store,Grocery Store,Skating Rink,Bakery,Pharmacy,Fast Food Restaurant,Mexican Restaurant,Café
91,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,2,Park,Italian Restaurant,Bank,Eastern European Restaurant,Harbor / Marina,Shopping Mall,Ice Cream Shop,Food & Drink Shop,Food,Ethiopian Restaurant
94,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,2,Park,Pizza Place,Bank,Fish & Chips Shop,Gym,Pharmacy,Clothing Store,Grocery Store,Restaurant,Hotel
96,M9L,North York,Humber Summit,43.756303,-79.565963,2,Electronics Store,Park,Pizza Place,Shopping Mall,Bakery,Pharmacy,Bank,Furniture / Home Store,Italian Restaurant,Flower Shop


In [46]:
toronto_merged[toronto_merged['Cluster Labels'] == 3].head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,3,Donut Shop,Zoo,Fish & Chips Shop,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant


In [47]:
toronto_merged[toronto_merged['Cluster Labels'] == 4].head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Coffee Shop,Trail,Arts & Crafts Store,Greek Restaurant,Park,Chinese Restaurant,Bank,Bakery,Supermarket
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Pizza Place,Fast Food Restaurant,Bank,Coffee Shop,Fried Chicken Joint,Automotive Shop,Food & Drink Shop,Sports Bar,Smoothie Shop,Breakfast Spot
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Indian Restaurant,Park,Coffee Shop,Pizza Place,Chinese Restaurant,Sandwich Place,Department Store,Mobile Phone Shop,Juice Bar,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Coffee Shop,Gas Station,Indian Restaurant,Pharmacy,Sandwich Place,Bank,Bakery,Deli / Bodega,Fast Food Restaurant,Chinese Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Ice Cream Shop,Sandwich Place,Indian Restaurant,Breakfast Spot,Grocery Store,Coffee Shop,Bowling Alley,Pizza Place,Train Station,Fish & Chips Shop
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,4,Coffee Shop,Chinese Restaurant,Grocery Store,Fast Food Restaurant,Discount Store,Pizza Place,Hockey Arena,Asian Restaurant,Sandwich Place,Bank
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,4,Grocery Store,Bus Line,Coffee Shop,Bakery,Trail,Bus Station,Mexican Restaurant,Metro Station,Fast Food Restaurant,Gas Station
10,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304,4,Electronics Store,Fast Food Restaurant,Restaurant,Coffee Shop,Pharmacy,Asian Restaurant,Bakery,Bank,Chinese Restaurant,Indian Restaurant
11,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,4,Middle Eastern Restaurant,Pizza Place,Bus Station,Hookah Bar,Asian Restaurant,Intersection,Chinese Restaurant,Korean Restaurant,Coffee Shop,Sandwich Place
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,4,Chinese Restaurant,Shopping Mall,Filipino Restaurant,Coffee Shop,Sandwich Place,Bakery,Caribbean Restaurant,Shanghai Restaurant,Cantonese Restaurant,Latin American Restaurant
